In [1]:
from img_xtend.utils.triton import TritonRemoteModel 


url = "http://localhost:8000/osnet_x1_0"

model = TritonRemoteModel(url)


In [2]:
import onnx

local_model = onnx.load("/home/nvidia/dev/img_new/models_weights/reidentification/osnet_x1_0.onnx")


In [3]:
import torch
device = 'cpu'

dummy_input = torch.randn(1, 3, 256, 128).to(device).half()


In [4]:
import onnxruntime
import numpy as np

device = 'cpu'

onnx_path = '/home/nvidia/dev/img_new/models_weights/reidentification/osnet_x1_0.onnx'

ort_session = onnxruntime.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])

dummy_input = np.random.randn(1, 3, 256, 128).astype(np.float16)
dummy_input = np.ones((1, 3, 256, 128)).astype(np.float16)
# dummy_input = torch.randn(1, 3, 256, 128).to(device).half()


def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
# ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(dummy_input)}
ort_inputs = {ort_session.get_inputs()[0].name: dummy_input}
ort_outs = ort_session.run(None, ort_inputs)

output_triton = model(dummy_input)[0]
print(output_triton.shape)
print(output_triton[0,:10])
print('output onnx')
print(type(ort_outs[0]))
print(ort_outs[0].shape)
print(ort_outs[0][0,:10])

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(output_triton, ort_outs[0],rtol=1e-1, atol=1e-01)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")


TypeError: 'NoneType' object is not subscriptable

In [5]:
path = "/home/nvidia/dev/img_new/models_weights/reidentification/osnet_x1_0_msmt17.pt"

model_torch = torch.load(path)

print(len(model_torch.keys()))

567


In [9]:
print(model_torch(dummy_input))

TypeError: 'collections.OrderedDict' object is not callable